In [ ]:
# install requirements to current kernel
%pip install -r requirements.txt

In [1]:
from components.balloon import Balloon, BalloonStatus
from components.network import Network
from components.networkanalyzer import NetworkAnalyzer as na
from components.plothelper import PlotHelper as ph
from components.windmap import WindMap as wm
from components.sample_antennas import *
from sims.antennas import *

import numpy as np
import random

In [12]:
reconfigs = []
meaningful = []

passes = 0

# generate a 500x500 wind map at 1u steps
MyWindMap = wm(500, 500, 1)
MyWindMap.create_windmap()
MyWindMap.bound(4) # bound map by 4 rows/cols around the border

MyPlotHelper = ph(MyWindMap)
MyPlotHelper.populate(show_scale=False)
# MyPlotHelper.plot(debug=False)

while passes < 3:
    # set up variables for dynamic number of balloons
    NumBalloons = 8
    Colors = ["red", "blue", "green", "orange", "purple", "red", "blue", "green", "orange", "purple"]
    Balloons = []
    GenericPath = []
    TickResponses = []
    AntennaToAdd = SaturnPatch
    
    for i in range(1, NumBalloons+1):
        Balloons.append(Balloon(MyWindMap, str(i), Colors[i-1]))
        GenericPath.append(str(i)) # no reconfiguration path: 1->2->3->4->5...
        start_pos_modifier = random.randint(-2, 2)
        Balloons[i-1].set_start_pos(-20 + start_pos_modifier, -45 + start_pos_modifier) # set start pos
        TickResponses.append(1) # balloons should tick until Balloon.tick() returns 0
        Balloons[i-1].add_antenna(AntennaToAdd)
    
    # MyPlotHelper.plot_balloons(B1, B2, B3, B4, B5)
    # MyPlotHelper.zoom_in(100, 100)
    
    NA = na(*Balloons)
    
    MyNetwork = Network(*Balloons,
                        start=Balloons[0], # start node
                        end=Balloons[NumBalloons-1], # end node
                        generic_path=GenericPath)
        
    for i in range(1, NumBalloons+1):
        while TickResponses[i-1]:
            TickResponses[i-1] = Balloons[i-1].tick()
    
    # check if all balloons hit the ground and are fall state
    sim_dud = False     
    for i in range(1, NumBalloons+1):
        if Balloons[i-1].current_status != BalloonStatus.FALL:
            sim_dud = True
            
    if sim_dud:
        print("\n == SIM DUD == \n")
        continue
            
    
    for i in range(1, 8000):
        MyNetwork.recalculate(i, NA.rp) # calculate new path
        MyNetwork.transmit(i, NA.rp) # determine meaningfulness
        
        if i % 1000 == 0:
            print("== TRANSMIT", passes, "PROGRESS: ", i, "/ 8000 ==")
    
    num_meaningfuls = len(list(filter(lambda x: x['meaningful'] == True, MyNetwork.reconfigures)))
    
    print("\n== SIMULATION", passes, "COMPLETE ==")
    print("Reconfigurations:", len(MyNetwork.reconfigures))
    reconfigs.append(len(MyNetwork.reconfigures))
    print("Meaningful (above threshhold) reconfigs:", num_meaningfuls)
    print("Percentage meaningful:", (num_meaningfuls/len(MyNetwork.reconfigures))*100, "\n")
    meaningful.append(num_meaningfuls)
    
    passes += 1

print("== ALL SIMS COMPLETE ==")
print("Mean reconfigurations:", np.mean(reconfigs))
print("Mean meaningful reconfigurations:", np.mean(meaningful))
print("St.dev of reconfigurations:", np.std(reconfigs))
print("Variance of meaningful reconfigurations:", np.std(meaningful))

1  popped at  6247.0
1  hit the ground at time t= 8508.0
2  popped at  7497.5
2  hit the ground at time t= 9763.0
3  popped at  6560.5
3  hit the ground at time t= 8827.5
4  popped at  7061.0
4  hit the ground at time t= 9325.0
5  popped at  7084.5
5  hit the ground at time t= 9347.5
6  popped at  7341.0
6  hit the ground at time t= 9602.5
7  popped at  7751.0
7  hit the ground at time t= 10015.0
8  popped at  6805.0
8  hit the ground at time t= 9070.0
== TRANSMIT 0 PROGRESS:  1000 / 8000 ==
== TRANSMIT 0 PROGRESS:  2000 / 8000 ==
== TRANSMIT 0 PROGRESS:  3000 / 8000 ==
== TRANSMIT 0 PROGRESS:  4000 / 8000 ==
== TRANSMIT 0 PROGRESS:  5000 / 8000 ==
== TRANSMIT 0 PROGRESS:  6000 / 8000 ==
== TRANSMIT 0 PROGRESS:  7000 / 8000 ==

== SIMULATION 0 COMPLETE ==
Reconfigurations: 15
Meaningful (above threshhold) reconfigs: 2
Percentage meaningful: 13.333333333333334 

1  popped at  7320.5
1  hit the ground at time t= 9582.5
2  popped at  6145.0
2  hit the ground at time t= 8410.5
3  popped at